## Topographic Complexity/Variability: Rugosity  
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 
* Du Preez, C. A new arc–chord ratio (ACR) rugosity index for quantifying three-dimensional landscape structural complexity. Landscape Ecol 30, 181–192 (2015). https://doi.org/10.1007/s10980-014-0118-8

### Initial setup

In [1]:
import numpy as np
import rasterio
import joblib
from scipy.linalg import lstsq
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

### Define data path

In [2]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
#input_file_name = 'Test_DEM.tif'
input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
#output_file_name = 'Test_pyRugosity.tif'
output_file_name = 'Tokeland_pyRugosity.tif'
#output_file_name = 'Nemah_pyRugosity.tif'
#output_file_name = 'Francies_pyRugosity.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [3]:
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [4]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

CRS: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",45.3333333333333],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",45.8333333333333],PARAMETER["standard_parallel_2",47.3333333333333],PARAMETER["false_easting",1640416.66666667],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
CRS as WKT: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitud

## Rugosity (slope-corrected)

Rugosity measures the complexity of the surface texture and is defined as the ratio of the actual surface area to the planar area. It is calculated using a $N \times N$ neighborhood around each pixel:

$$
\text{Rugosity Index} = \frac{\text{surface area of NxN neighborhood}}{\text{planar area of NxN neighborhood}}
$$

, where $N$ is the window size. In practice, the surface area is estimated using the gradients of the elevation within the neighborhood, accounting for the additional area contributed by the terrain's slope.  

The planer area should be the area of the surface orthogonally projected onto a plane of best fit within the window. To calculate the planar area as the area of the surface orthogonally projected onto a plane of best fit within the window, we need to compute the local slope and aspect for each cell within the window, and then use these to project the area onto the best-fit plane. After this local slope correction, the result is the indext so-called arc-chord ratio (ACR) rugosity index. See details in Du Preez (2015).

##### Rugosity functions (optimized with chunk processing and parallel processing) 

In [5]:
def fit_plane_to_window(window):
    """
    Fit a plane to a window of elevation data.

    :param window: 2D array of elevation values.
    :return: Coefficients of the plane.
    """
    window_size = window.shape[0]
    x, y = np.indices(window.shape)
    A = np.c_[x.ravel(), y.ravel(), np.ones(window.size)]
    C, _, _, _ = lstsq(A, window.ravel())  # Coefficients of the plane
    return C

In [6]:
def calculate_orthogonal_projected_area(window, coeffs):
    """
    Calculate the area of the surface orthogonally projected onto a plane of best fit.

    :param window: 2D array of elevation values.
    :param coeffs: Coefficients of the plane.
    :return: Projected area of the surface.
    """
    # Calculate normal vector to the plane
    nx, ny, nz = coeffs[0], coeffs[1], -1
    normal = np.array([nx, ny, nz])
    normal_length = np.linalg.norm(normal)
    
    # Calculate area of the projected plane
    window_size = window.shape[0]
    projected_area = window_size**2 / normal_length
    return projected_area

In [7]:
def calculate_rugosity_chunk(dem, window_size, i, j, chunk_size, overlap):
    """
    Calculate rugosity for a chunk of the raster, considering an overlap to avoid edge artifacts.
    
    :param dem: 2D array of elevation values for the entire raster.
    :param window_size: Size of the moving window to calculate rugosity.
    :param i: Starting row index for the chunk.
    :param j: Starting column index for the chunk.
    :param chunk_size: Size of the chunks to divide the raster for parallel processing.
    :param overlap: Width of the overlap area around each chunk.
    :return: 2D array of rugosity values for the chunk.
    """
    # Define the extended chunk indices with overlap
    start_i = max(i - overlap, 0)
    end_i = min(i + chunk_size + overlap, dem.shape[0])
    start_j = max(j - overlap, 0)
    end_j = min(j + chunk_size + overlap, dem.shape[1])
    
    # Extract the extended chunk from the DEM
    chunk = dem[start_i:end_i, start_j:end_j]
    
    # Calculate rugosity for the extended chunk
    rugosity_chunk = np.zeros((end_i - start_i, end_j - start_j), dtype=np.float32)
    for row in range(overlap, end_i - start_i - overlap):
        for col in range(overlap, end_j - start_j - overlap):
            window = chunk[row - overlap:row + overlap + 1, col - overlap:col + overlap + 1]
            coeffs = fit_plane_to_window(window)
            projected_area = calculate_orthogonal_projected_area(window, coeffs)
            surface_area = np.sum(np.sqrt(1 + np.gradient(window)[0]**2 + np.gradient(window)[1]**2))
            rugosity_chunk[row, col] = surface_area / projected_area
    
    # Return the non-overlapping part of the chunk
    return rugosity_chunk[overlap:-overlap, overlap:-overlap], (i, j)

In [8]:
def compute_rugosity_parallel(dem, window_size, chunk_size):
    """
    Compute rugosity for the raster using parallel processing with chunk-based approach.
    
    :param dem: 2D array of elevation values.
    :param window_size: Size of the moving window to calculate rugosity.
    :param chunk_size: Size of the chunks to divide the raster for parallel processing.
    :return: 2D array of rugosity values.
    """
    rows, cols = dem.shape
    overlap = window_size // 2  # Set the overlap width to half of the window size
    rugosity_map = np.full((rows, cols), np.nan, dtype=np.float32)

    # Define chunk indices for parallel processing
    chunk_indices = [(i, j)
                     for i in range(overlap, rows - overlap, chunk_size)
                     for j in range(overlap, cols - overlap, chunk_size)]
    
    # Process chunks in parallel
    with Parallel(n_jobs=-1) as parallel:
        results = parallel(delayed(calculate_rugosity_chunk)(dem, window_size, i, j, chunk_size, overlap)
                           for i, j in tqdm(chunk_indices, desc='Computing Rugosity'))

    # Stitch the results together
    for (rugosity_chunk, (i, j)) in results:
        # Compute the correct slice size for the rugosity map
        slice_size_i = rugosity_chunk.shape[0]
        slice_size_j = rugosity_chunk.shape[1]
        rugosity_map[i:i + slice_size_i, j:j + slice_size_j] = rugosity_chunk

    return rugosity_map

Calculate Rugosity with a given window size

In [9]:
# Calculate rugosity for the entire DEM with a specified window size
window_size = 3  # N x N neighborhood
chunk_size = 512  # Example chunk size, adjust based on your system's memory capacity

# Compute rugosity with parallel processing and chunk-based approach
rugosity_map = compute_rugosity_parallel(dem, window_size, chunk_size)

Computing Rugosity:   0%|          | 0/2538 [00:00<?, ?it/s]

Output data into GeoTIFF
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [10]:
# Update metadata for output GeoTIFF
meta.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')

# Write Rugosity to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
   dst.write(rugosity_map.astype(rasterio.float32), 1)